# Install nnU-Netv1

There will be an issue with argparse. Just click `Restart Session` and then you can go on with the next section.

In [ ]:
!pip install git+https://github.com/MIC-DKFZ/nnUNet.git@nnunetv1

# Check available models

In [ ]:
!nnUNet_print_available_pretrained_models

# Set environment variable

In [ ]:
%env RESULTS_FOLDER=RESULTS_FOLDER
!mkdir RESULTS_FOLDER
!mkdir RESULTS_FOLDER/nnUNet

# Download pretrained model

In [ ]:
# This is how you would normally do it but this does not work :(
# !nnUNet_download_pretrained_model Task004_Hippocampus

In [ ]:
!wget -O ./RESULTS_FOLDER/nnUNet/Task004_Hippocampus.zip https://zenodo.org/record/4003545/files/Task004_Hippocampus.zip?download=1

In [ ]:
!unzip ./RESULTS_FOLDER/nnUNet/Task004_Hippocampus.zip -d ./RESULTS_FOLDER/nnUNet

# Download data

In [ ]:
!wget -O Task04_Hippocampus.tar https://drive.google.com/uc?id=1RzPB1_bqzQhlWvU-YGvZzhx2omcDh38C&export=download

In [ ]:
!tar -xvf Task04_Hippocampus.tar

# View data sample

In [ ]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
!pip install celluloid
from celluloid import Camera
from IPython.display import HTML

In [ ]:
sample_number = '001'

In [ ]:
image_1 = nib.load(f'./Task04_Hippocampus/imagesTr/hippocampus_{sample_number}.nii.gz')
mask_1 = nib.load(f'./Task04_Hippocampus/labelsTr/hippocampus_{sample_number}.nii.gz')

image_1_arr = image_1.get_fdata()
mask_1_arr = mask_1.get_fdata()

print('Image shape:', image_1_arr.shape)
print('Mask shape:', mask_1_arr.shape)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(7, 5))
camera = Camera(fig)
axs = axs.ravel()

for slice_number in range(image_1_arr.shape[1]):
    axs[0].imshow(image_1_arr[:, slice_number, :], cmap='gray')
    axs[0].set_title('Input Image')
    axs[0].axis('off')

    axs[1].imshow(mask_1_arr[:, slice_number, :], cmap='gray')
    axs[1].set_title('True Mask')
    axs[1].axis('off')

    axs[2].imshow(image_1_arr[:, slice_number, :], cmap='gray')
    axs[2].imshow(mask_1_arr[:, slice_number, :], cmap='gray', alpha=0.4)
    axs[2].set_title('Overlaid Mask')
    axs[2].axis('off')

    plt.ioff()
    camera.snap()

In [ ]:
animation_before = camera.animate()
HTML(animation_before.to_html5_video())

In [ ]:
!mkdir ./Task04_Hippocampus/SUBSET_IN
!mkdir ./Task04_Hippocampus/SUBSET_OUT
!cp ./Task04_Hippocampus/imagesTr/hippocampus_{sample_number}.nii.gz ./Task04_Hippocampus/SUBSET_IN
!mv ./Task04_Hippocampus/SUBSET_IN/hippocampus_{sample_number}.nii.gz ./Task04_Hippocampus/SUBSET_IN/hippocampus_{sample_number}_0000.nii.gz

# Run segmentation model

In [ ]:
!nnUNet_predict -i ./Task04_Hippocampus/SUBSET_IN/ -o ./Task04_Hippocampus/SUBSET_OUT -t Task004_Hippocampus -m 3d_fullres

# View results

In [ ]:
image_1 = nib.load(f'./Task04_Hippocampus/imagesTr/hippocampus_{sample_number}.nii.gz')
mask_1 = nib.load(f'./Task04_Hippocampus/labelsTr/hippocampus_{sample_number}.nii.gz')
pred_1 = nib.load(f'./Task04_Hippocampus/SUBSET_OUT/hippocampus_{sample_number}.nii.gz')

image_1_arr = image_1.get_fdata()
mask_1_arr = mask_1.get_fdata()
pred_1_arr = pred_1.get_fdata()

print('Image shape:', image_1_arr.shape)
print('Mask shape:', mask_1_arr.shape)
print('Predicted mask shape:', pred_1_arr.shape)

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(7, 4.5))
camera = Camera(fig)
axs = axs.ravel()

for slice_number in range(image_1_arr.shape[1]):
    axs[0].imshow(image_1_arr[:, slice_number, :], cmap='gray')
    axs[0].set_title('Input Image')
    axs[0].axis('off')

    axs[1].imshow(mask_1_arr[:, slice_number, :], cmap='gray')
    axs[1].set_title('True Mask')
    axs[1].axis('off')

    axs[2].imshow(image_1_arr[:, slice_number, :], cmap='gray')
    axs[2].imshow(mask_1_arr[:, slice_number, :], cmap='gray', alpha=0.4)
    axs[2].set_title('Overlaid True Mask')
    axs[2].axis('off')

    axs[3].axis('off')

    axs[4].imshow(pred_1_arr[:, slice_number, :], cmap='gray')
    axs[4].set_title('Predicted Mask')
    axs[4].axis('off')

    axs[5].imshow(image_1_arr[:, slice_number, :], cmap='gray')
    axs[5].imshow(pred_1_arr[:, slice_number, :], cmap='gray', alpha=0.4)
    axs[5].set_title('Overlaid Predicted Mask')
    axs[5].axis('off')

    plt.ioff()
    camera.snap()

In [ ]:
animation_after = camera.animate()
HTML(animation_after.to_html5_video())